In [1]:
import os
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

In [2]:
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix

In [11]:
from sklearn.utils import shuffle


### Loading Model

In [3]:
import pickle
filename = 'XGB_model.model'
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, Y_test)
# print(result)

In [4]:
def fold(sound_dataframe, sets_loc):
    big = []
    for j in os.listdir(sets_loc):      

        file = open(sets_loc+j, 'r') 
        Lines = file.readlines() 

        count = 0
        # Strips the newline character 
        # for line in Lines: 
        #     print("Line{}: {}".format(count, line.strip())) 
        try:
            set_dataframe = pd.DataFrame()
            for i, line in enumerate(Lines):
                A = sound_dataframe[(sound_dataframe['73'] == (Lines[i].strip()+".json"))]
                set_dataframe = pd.DataFrame.append(set_dataframe,A)
        except:
            set_dataframe = pd.DataFrame()
            for i, line in enumerate(Lines):
                A = sound_dataframe[(sound_dataframe[73] == (Lines[i].strip()+".json"))]
                set_dataframe = pd.DataFrame.append(set_dataframe,A)
            
        
        big.append(set_dataframe)
    fold_1 = shuffle(pd.concat(big[:6]))
    test_1 = shuffle(big[6])
    fold_2 = shuffle(pd.concat(big[1:7]))
    test_2 = shuffle(big[0])
    fold_3 = shuffle(pd.concat([big[0],big[2],big[3],big[4], big[5], big[6]]))
    test_3 = shuffle(big[1])
    fold_4 = shuffle(pd.concat([big[0],big[1],big[3],big[4], big[5], big[6]]))
    test_4 = shuffle(big[2])
    fold_5 = shuffle(pd.concat([big[0],big[2],big[1],big[4], big[5], big[6]]))
    test_5 = shuffle(big[3])
    fold_6 = shuffle(pd.concat([big[0],big[2],big[1],big[3], big[5], big[6]]))
    test_6 = shuffle(big[4])
    fold_7 = shuffle(pd.concat([big[0],big[2],big[1],big[4], big[3], big[6]]))
    test_7 = shuffle(big[5])
    
    return fold_1,test_1,fold_2,test_2,fold_3,test_3,fold_4,test_4,fold_5,test_5, fold_6, test_6, fold_7,test_7

### Loading CSV

In [5]:
train_dataframe = pd.read_csv('./Train_XGB.csv')
eko_dataframe = pd.read_csv("./eko.csv")
mic_dataframe = pd.read_csv("./mic.csv")

In [9]:
sets_loc = './Sets/'

### Folds

In [12]:
f1,t1,f2,t2,f3,t3,f4,t4,f5,t5,f6,t6,f7,t7 = fold(mic_dataframe,sets_loc)
fold_list_mic = [f1,t1,f2,t2,f3,t3,f4,t4,f5,t5,f6,t6,f7,t7]
f1,t1,f2,t2,f3,t3,f4,t4,f5,t5,f6,t6,f7,t7 = fold(eko_dataframe,sets_loc)
fold_list_eko = [f1,t1,f2,t2,f3,t3,f4,t4,f5,t5,f6,t6,f7,t7]

In [17]:
list_test = []
def test_wheeze(test_set):
    X_test = test_set.iloc[:,:72]
    y_test = test_set.iloc[:,72]
    scaler = StandardScaler()
    scaler.fit(train_dataframe.iloc[:,:72])
    X_test = scaler.transform(X_test)
    X_test = pd.DataFrame(X_test)
    return metrics.accuracy_score(y_test,loaded_model.predict(X_test))

### Testing EKO (7Folds)

In [24]:
acc_eko_wheeze = []
for i in range(0,len(fold_list_eko),2):
    acc_eko_wheeze.append(test_wheeze(fold_list_eko[i+1]))

In [25]:
acc_eko_wheeze

[0.6090047393364929,
 0.630901287553648,
 0.47820672478206727,
 0.506872852233677,
 0.45776566757493187,
 0.4140625,
 0.5979073243647235]

In [ ]:
acc_eko_wheeze.append(np.mean(acc_eko_wheeze))

In [29]:
pd.DataFrame(acc_eko_wheeze, index=['Fold1', 'Fold2', 'Fold3', 'Fold4', 'Fold5', 'Fold6', 'Fold7', 'Mean'])

,0
Fold1,0.609005
Fold2,0.630901
Fold3,0.478207
Fold4,0.506873
Fold5,0.457766
Fold6,0.414062
Fold7,0.597907
Mean,0.527817


### Testing MIC

In [26]:
acc_mic_wheeze = []
for i in range(0,len(fold_list_mic),2):
    acc_mic_wheeze.append(test_wheeze(fold_list_mic[i+1]))

In [27]:
acc_mic_wheeze

[0.4277251184834123,
 0.4220314735336195,
 0.5466998754669987,
 0.5652920962199313,
 0.5599455040871935,
 0.603515625,
 0.4469357249626308]

In [30]:
acc_mic_wheeze.append(np.mean(acc_mic_wheeze))

In [31]:
pd.DataFrame(acc_mic_wheeze, index=['Fold1', 'Fold2', 'Fold3', 'Fold4', 'Fold5', 'Fold6', 'Fold7', 'Mean'])

,0
Fold1,0.427725
Fold2,0.422031
Fold3,0.546700
Fold4,0.565292
Fold5,0.559946
Fold6,0.603516
Fold7,0.446936
Mean,0.510306
